# Cowboy results for X

***Add notes about your experiment here.***

*NB* requires a kernel with pycorn. 

If you don't have one, you an use 'shifty'

To get shifty as a kernal option, log onto the digs and run:
/software/containers/scripts/registerapptainerkernel.sh /software/containers/shifty.sif

In [1]:
import glob
from datetime import date

today = date.today().strftime("%Y%m%d")[2:]
print(f"Today: {today}")
# import warnings; warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from scipy.cluster import hierarchy
import sys, os

#sys.path.append("/net/software/lab/cowboy")
from wetlab_utils_spr_3 import *
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import seaborn as sns

sns.set_theme(
    context="talk",
    palette="colorblind",
    style="ticks",
    rc={"axes.spines.right": False, "axes.spines.top": False},
)

Today: 250121


In [17]:
df.columns

Index(['Destination Well', 'eblock', 'Destination Plate Name', 'Name',
       'Vector', 'plasmid_seq', 'ORF', 'exp_prod', 'nAA', 'protomer_MW',
       'aliphatic_idx', 'e280', 'OD280', 'e205', 'OD205', 'nC', 'charge@8.0',
       'pI', 'MW', 'culture_vol', 'culture_time', 'culture_temp',
       'sec_instrument', 'sec_column', 'sec_buffer', 'category', 'vol280',
       'vol260', 'vol230', 'vol205', 'A280', 'A260', 'A230', 'A205',
       'trace_id', 'fractions', 'frac_vol', 'frac_start_stop', 'flow_rate',
       'vol', 'Abs', 'vol_clipped', 'Abs_corr', 'Abs_norm', 'main_peak',
       'main_peak_height', 'tot_integral', 'nPeaks', 'monodisperse', 'peaks',
       'peak_heights', 'light_idx', 'vol_light', 'Abs_light', 'Abs_norm_light',
       'well_id', 'path_length', 'tot_yield', 'yield_per_Leq',
       'tot_yield_from_205nm', 'yield_per_Leq_from_205nm',
       'expected_norm_retention', 'expected_Vel', 'main_peak_norm_retention',
       'main_peak_estimated_MW', 'main_peak_agg_state', 'corr

In [18]:
for w, r in df.set_index("well_id").iterrows():
    print (f'{w}: e280 {r.e280} MW: {r.protomer_MW}')

1_A1: e280 17990 MW: 10923.961440000012
1_A3: e280 11000 MW: 11301.227840000009
1_A4: e280 19940 MW: 12792.129240000011
1_A5: e280 8480 MW: 12575.167740000012
1_A6: e280 8480 MW: 14170.090240000014
1_A7: e280 8480 MW: 15046.125040000008
1_A8: e280 11460 MW: 16980.67654000002
1_A9: e280 6990 MW: 13166.500940000005
1_A10: e280 9970 MW: 13127.065440000013
1_A11: e280 8480 MW: 12447.772940000012
1_A12: e280 11460 MW: 13537.193840000014
1_B1: e280 13980 MW: 12977.22834000001
1_B2: e280 13980 MW: 13095.528540000014
1_B3: e280 13980 MW: 13688.50664000001
1_B4: e280 8480 MW: 13447.100540000007
1_B5: e280 18450 MW: 13820.518340000017
1_B6: e280 11460 MW: 13475.072740000014
1_B7: e280 13980 MW: 14288.966340000015
1_B8: e280 6990 MW: 13632.175940000005
1_B9: e280 8480 MW: 13844.521840000012
1_B10: e280 17990 MW: 14192.53384000001
1_B11: e280 8480 MW: 14376.001640000013
1_B12: e280 9970 MW: 14531.174340000012
1_C1: e280 6990 MW: 14404.18844000001
1_C2: e280 13980 MW: 14989.570240000012
1_C3: e280 

## Correct concentration or continue

In [19]:
#read in csv from nanodrop
#or continue with calculated concentrations
nanodrop_df = pd.read_csv("test.txt",sep='\t')
nanodrop_df

,Plate ID,Well,Sample ID,User ID,Date,Time,conc.,units,260/280,A280 10mm,E 1%,Ext. Coeff x10e3,Mol. Wt. kDa,Cursor Pos.,Cursor abs.
0,NaN,A1,NaN,Default,10/15/2024,1:41 PM,0.002744,mg/ml,6.82,0.003,10.0,1.0,1.0,280,0.003
1,NaN,B1,NaN,Default,10/15/2024,1:41 PM,0.168600,mg/ml,0.63,0.169,10.0,1.0,1.0,280,0.169
2,NaN,C1,NaN,Default,10/15/2024,1:41 PM,0.010740,mg/ml,1.33,0.011,10.0,1.0,1.0,280,0.011
3,NaN,D1,NaN,Default,10/15/2024,1:41 PM,0.032020,mg/ml,0.37,0.032,10.0,1.0,1.0,280,0.032
4,NaN,E1,NaN,Default,10/15/2024,1:41 PM,0.024870,mg/ml,0.73,0.025,10.0,1.0,1.0,280,0.025
5,NaN,F1,NaN,Default,10/15/2024,1:41 PM,0.158700,mg/ml,0.66,0.159,10.0,1.0,1.0,280,0.159
6,NaN,G1,NaN,Default,10/15/2024,1:41 PM,0.031070,mg/ml,0.60,0.031,10.0,1.0,1.0,280,0.031
7,NaN,H1,NaN,Default,10/15/2024,1:41 PM,0.066390,mg/ml,0.66,0.066,10.0,1.0,1.0,280,0.066
8,NaN,A2,NaN,Default,10/15/2024,1:43 PM,0.086410,mg/ml,0.38,0.086,10.0,1.0,1.0,280,0.086
9,NaN,B2,NaN,Default,10/15/2024,1:43 PM,0.079040,mg/ml,0.47,0.079,10.0,1.0,1.0,280,0.079


In [26]:
M = .169/13980
mM = M * 1000
mg_ml = M * 12479
mg_ml

0.15085486409155938

## generate SPR file
- pool fractions first
- `highest_conc_nM`: concentration of the highest concentration well in your dilution series in nM
- `volume_uL`: how many ul in each well of your 384 well plate
- `dilution_step`: divide your conc by this number for the next concentration. 
- `num_dilutions`: how many dilutions do you want for each sample

i.e. if 
highest_conc_nM = 1000
volume_uL = 150
dilution_step = 5 
num_dilutions = 4

then you will end up with plates with 150 ul in them at 1000, 200, 40, and 8 nM

In [20]:
highest_conc_nM = 2000 #nM
volume_uL = 150 #uL
dilution_step = 5 #divide your conc by this number for the next concentration. i.e if dilution_step1000, 
num_dilutions = 4
plate = 384 #384 or 96
checkpoint = True #do you want to dump the spr_df so you can just read that in?

spr_df = make_spr_df(df, highest_conc_nM, volume_uL, dilution_step, num_dilutions, checkpoint=checkpoint)
spr_df

Total samples: 47


,well_id,conc_uM,pooled_vol,eblock,Vector,conc_nM,dilutions,dilution_step,volume_step,highest_conc_nM
0,1_A1,2.0,1.500000,TGAGGCAGCTTTGCACTTGGCTGAGAGATTGTTGTCAATACTACGG...,LM0627,2000.0,4,5,37.5,2000
2,1_A3,2.0,0.822739,ATACTACGGTCTCAAGGAAGTGGCGGCAGCCTGGCCGAGGAGGTGC...,LM0627,2000.0,4,5,37.5,2000
3,1_A4,2.0,1.500000,ATACTACGGTCTCAAGGAAGTGGCGGTAGCAAAGTGGAAGAAGCCA...,LM0627,2000.0,4,5,37.5,2000
4,1_A5,2.0,1.500000,ATACTACGGTCTCAAGGAAGCGGCGGTGAAGAACTGAAAAAGAAAT...,LM0627,2000.0,4,5,37.5,2000
5,1_A6,2.0,0.481237,ATACTACGGTCTCAAGGAAGCGGCGGTCTGGACCCGAAAGCGCGTG...,LM0627,2000.0,4,5,37.5,2000
6,1_A7,2.0,1.500000,ATACTACGGTCTCAAGGAAGCGGCGGTATGCCAAAAGAAGAACGTA...,LM0627,2000.0,4,5,37.5,2000
7,1_A8,2.0,1.500000,ATACTACGGTCTCAAGGAAGCGGTGGCGAAGAACTGCGTGAATTAC...,LM0627,2000.0,4,5,37.5,2000
8,1_A9,2.0,1.500000,ATACTACGGTCTCAAGGAAGTGGCGGTGAAGAGGAAGAAATTGAAC...,LM0627,2000.0,4,5,37.5,2000
9,1_A10,2.0,1.500000,ATACTACGGTCTCAAGGAAGTGGCGGCATGAAGGAGAAAGTGCTGG...,LM0627,2000.0,4,5,37.5,2000
10,1_A11,2.0,0.264624,ATACTACGGTCTCAAGGAAGTGGTGGCAGCCCAGCCGCCCGTGCCG...,LM0627,2000.0,4,5,37.5,2000


In [21]:
#spr_df = pd.read_csv("spr_readin_df_ckpt.csv")
spr_df = calculate_conc_and_vol_and_drop_samples(spr_df)

Before calculating conc and volume: 47 samples
100.0% of wells will have the desired concentration.
WARNING! 1_D12 only has 133.49 uL. Amount only allows 106.79400020630369 uL per well. Skipping well.
After removing samples that cannot be normalized properly: 46 samples


In [22]:
#generate csv for spr
spr_df = write_spr_csv(spr_df,"test_def_spr", plate)

In [23]:
gen_spr_ot2_script(
    spr_df,
    plate,
    #path="/home/ach94/projects/p05_peptide_ligase/e151_ligase_cowboy/cowboy/",
    template="/home/srgerb/scripts/cowboy/spr/SPR_norm_and_dilute.py",
)


[ ['1_A1',], [187.5], 1, 'A1', 0.000],
[ ['1_A3',], [187.5], 1, 'C1', 0.000],
[ ['1_A4',], [187.5], 1, 'E1', 0.000],
[ ['1_A5',], [187.5], 1, 'G1', 0.000],
[ ['1_A6',], [187.5], 1, 'I1', 0.000],
[ ['1_A7',], [187.5], 1, 'K1', 0.000],
[ ['1_A8',], [187.5], 1, 'M1', 0.000],
[ ['1_A9',], [187.5], 1, 'O1', 0.000],
[ ['1_A10',], [187.5], 1, 'B1', 0.000],
[ ['1_A11',], [187.5], 1, 'D1', 0.000],
[ ['1_A12',], [187.5], 1, 'F1', 0.000],
[ ['1_B1',], [187.5], 1, 'H1', 0.000],
[ ['1_B2',], [187.5], 1, 'J1', 0.000],
[ ['1_B3',], [187.5], 1, 'L1', 0.000],
[ ['1_B4',], [187.5], 1, 'N1', 0.000],
[ ['1_B5',], [187.5], 1, 'P1', 0.000],
[ ['1_B6',], [187.5], 1, 'A5', 0.000],
[ ['1_B7',], [187.5], 1, 'C5', 0.000],
[ ['1_B8',], [187.5], 1, 'E5', 0.000],
[ ['1_B9',], [187.5], 1, 'G5', 0.000],
[ ['1_B10',], [187.5], 1, 'I5', 0.000],
[ ['1_B11',], [187.5], 1, 'K5', 0.000],
[ ['1_B12',], [187.5], 1, 'M5', 0.000],
[ ['1_C1',], [187.5], 1, 'O5', 0.000],
[ ['1_C2',], [187.5], 1, 'B5', 0.000],
[ ['1_C3',], [187.5